In [ ]:
#Iport packages
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pixellib
from pixellib.torchbackend.instance import instanceSegmentation
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#Input folder and add full path
image_filename = 'data/'
img = load_img(image_filename)
image_data = img_to_array(img)
images_data = np.expand_dims(image_data, axis=0)
datagen = ImageDataGenerator(width_shift_range=0.2)
train_generator = datagen.flow(images_data, batch_size=1)
rows = 5
columns = 4
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation  
datagen = ImageDataGenerator(height_shift_range=0.2)
train_generator = datagen.flow(images_data, batch_size=1)
rows = 5
columns = 4
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation for horizontal_flip
datagen = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen.flow(images_data, batch_size=1)
rows = 2
columns = 2
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):        
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation for vertical_flip
datagen = ImageDataGenerator(vertical_flip=True)
train_generator = datagen.flow(images_data, batch_size=1)
rows = 2
columns = 2
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation for rotation
datagen = ImageDataGenerator(rotation_range=50)
train_generator = datagen.flow(images_data)
rows = 5
columns = 4
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation for brightness
datagen = ImageDataGenerator(brightness_range=[0.15,2.0])
train_generator = datagen.flow(images_data, batch_size=1)
rows = 5
columns = 4
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation for zoom
datagen = ImageDataGenerator(zoom_range=[5,0.5])
train_generator = datagen.flow(images_data, batch_size=1)
rows = 5
columns = 4
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#Data Augumentation for zoom
datagen = ImageDataGenerator(zoom_range=[5,0.5])
train_generator = datagen.flow(images_data, batch_size=1)
rows = 5
columns = 4
fig, axes = plt.subplots(rows,columns)
for r in range(rows):
    for c in range(columns):
        image_batch = train_generator.next()
        image = image_batch[0].astype('uint8')
        axes[r,c].imshow(image)
fig.set_size_inches(15,10)

In [ ]:
#train data
train_images = glob.glob("data/*")

In [ ]:
train_images[:10]

In [ ]:
#Apply Segmentation using threshol segmentation
def crop_coords(img):
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)    
    cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(cnts, key = cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    return (x, y, w, h)
def truncation_normalization(img):
    
    Pmin = np.percentile(img[img!=0], 5)
    Pmax = np.percentile(img[img!=0], 99)
    truncated = np.clip(img,Pmin, Pmax)  
    normalized = (truncated - Pmin)/(Pmax - Pmin)
    normalized[img==0]=0
    return normalized


def clahe(img, clip):
    
    clahe = cv2.createCLAHE(clipLimit=clip)
    cl = clahe.apply(np.array(img*255, dtype=np.uint8))
    return cl

In [ ]:
_, axs = plt.subplots(2, 5, figsize=(16, 8))
axs = axs.flatten()
images = []
for img_path, ax in zip(train_images[:10], axs):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    (x, y, w, h) = crop_coords(img)
    # Create a Rectangle patch
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    img_cropped = img[y:y+h, x:x+w]
    images.append(img_cropped)
    ax.imshow(img, cmap="bone")
plt.savefig("rectangles.png")
plt.show()

In [ ]:
_, axs = plt.subplots(2, 5, figsize=(16, 8))
axs = axs.flatten()
final_imgs = []
IMG_SIZE = 512
for img_cropped, ax in zip(images, axs):
    img_normalized = truncation_normalization(img_cropped)
    cl1 = clahe(img_normalized, 1.0)
    cl2 = clahe(img_normalized, 2.0)
    img_final = cv2.merge((np.array(img_normalized*255, dtype=np.uint8),cl1,cl2))
    img_final = cv2.resize(img_final, (IMG_SIZE, IMG_SIZE))
    ax.imshow(img_final)
    
plt.savefig("final_imgs.png")
plt.show()

In [ ]:
#Apply Segmentation using Canny segmentation
img = cv2.imread(r'DSC_0010.jpg')
b,g,r = cv2.split(img)
rgb_img = cv2.merge([r,g,b])
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
kernel = np.ones((2,2),np.uint8)
closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel, iterations = 2)
sure_bg = cv2.dilate(closing,kernel,iterations=3)
dist_transform = cv2.distanceTransform(sure_bg,cv2.DIST_L2,3)
ret, sure_fg = cv2.threshold(dist_transform,0.1*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers+1
markers[unknown==255] = 0
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]
plt.subplot(211),plt.imshow(rgb_img)
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(212),plt.imshow(thresh, 'gray')
plt.imsave(r'thresh.png',thresh)
plt.title("Otsu's binary threshold"), plt.xticks([]), plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
#Segmentation Model Creation
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=3,padding="same",activation="relu",input_shape=(50,50,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(200,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

In [ ]:
#Apply Segmentation using instance segmentation
ins = instanceSegmentation()
ins.load_model("pointrend_resnet50.pkl", detection_speed = "fast")
ins.segmentImage("DSC_0010.jpg", show_bboxes=True, output_image_name="output_image.jpg")

In [ ]:
ins = instanceSegmentation()
ins.load_model("pointrend_resnet50.pkl")
target_classes = ins.select_target_classes(person = True)
ins.segmentImage("DSC_0010.jpg", show_bboxes=True, segment_target_classes = target_classes, output_image_name="output_image1.jpg")

In [ ]:
ins = instanceSegmentation()
ins.load_model("pointrend_resnet50.pkl")
ins.segmentImage("DSC_0010.jpg", show_bboxes=True, extract_segmented_objects=True, extract_from_box = True,
save_extracted_objects=True, output_image_name="output_image2.jpg" )

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import *
from sklearn.metrics import *
from tensorflow.keras.models import load_model
input_path = 'data/'

In [ ]:
learning_rate = 0.001
buffer_size = 512
batch_size = 16
epochs = 2

img_size = 224

In [ ]:
#Read directory
def read_directory():
    data_filenames = []
    data_labels = []

    for filename in os.listdir(input_path + '1'):
        data_filenames.append(input_path + '1/' + filename)
        data_labels.append(0)

    for filename in os.listdir(input_path + '2'):
        data_filenames.append(input_path + '2/' + filename)
        data_labels.append(1)

    for filename in os.listdir(input_path + '3'):
        data_filenames.append(input_path + '3/' + filename)
        data_labels.append(2)
        
    data_size = len(data_labels)

    tmp_uni = list(zip(data_filenames, data_labels))

    random.shuffle(tmp_uni)

    train_size = int(data_size * train_percentage)
    print('Size of training set：', train_size)
    print('Size of test set：', data_size - train_size)

    train_list = tmp_uni[0:train_size]
    test_list = tmp_uni[train_size:]

    train_filenames, train_labels = zip(*train_list)
    test_filenames, test_labels = zip(*test_list)

    return train_filenames, train_labels, test_filenames, test_labels

In [ ]:
def build_train_tfrecord(train_filenames, train_labels):  
    with tf.io.TFRecordWriter(train_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())


def build_test_tfrecord(test_filenames, test_labels):  
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(test_filenames, test_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())


In [ ]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']


def get_train_dataset(train_tfrecord):  # read TFRecord
    raw_train_dataset = tf.data.TFRecordDataset(train_tfrecord)
    train_dataset = raw_train_dataset.map(_parse_example)

    return train_dataset


def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset


def data_Preprocessing(train_dataset, test_dataset):
    train_dataset = train_dataset.shuffle(buffer_size)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, test_dataset

In [ ]:
#Apply transfer Learning using CNN
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[3, 3],
            padding='same',
            activation=tf.nn.relu,
            input_shape=(img_size, img_size, 3)
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(int(img_size/4) * int(img_size/4) * 64,))
        self.dense1 = tf.keras.layers.Dense(units=512, activation=tf.nn.relu)
        self.drop1 = tf.keras.layers.Dropout(0.5)
        self.dense2 = tf.keras.layers.Dense(units=3, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.drop1(x)
        output = self.dense2(x)
        return output

In [ ]:
#create and defing VGG16_mode
def VGG16_model():
    vgg16 = tf.keras.applications.VGG16(weights='imagenet',include_top=False, input_shape=[img_size,img_size,3])
    vgg16.trainable= True
    model = tf.keras.Sequential([
        vgg16,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )
    
    return model

In [ ]:
def scheduler(epoch,lr):
    if epoch<8:
        return lr
    else:
        return lr*0.9
callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

In [ ]:
# define and schedule all models
def train():
    time_start = time.time()
    model = VGG16_model()
   
    
    model.summary()
    
    train_history = model.fit(train_dataset, epochs=epochs, callbacks=[callback])

    model.save('mymodel.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return train_history